In [ ]:
!pip install faker 

In [ ]:
!pip install tqdm

## Modelo simples para gerar dados aleatórios com faker

In [7]:
# Gera dados aleatórios 
import faker
import os
import sys
import time
from tqdm import tqdm
import random

# Cria um objeto Faker
fake = faker.Faker()

def format_time(segundos):
    """
    Formata os milisegundos em hora:minuto:segundo
    """
    if segundos < 60:
        return f"{segundos:.3f} segundos"
    elif segundos < 3600:
        minutos, segundos = divmod(segundos, 60)
        return f"{int(minutos)} minutos {int(segundos)} segundos"
    else:
        horas, remainder = divmod(seconds, 3600)
        minutos, segundos = divmod(remainder, 60)
        if minutos == 0:
            return f"{int(horas)} horas {int(segundos)} segundos"
        else:
            return f"{int(horas)} horas {int(minutos)} minutos {int(segundos)} segundos"

# Função para gerar os dados de teste
def generate_test_data(num_rows):
  """
  Gera dados de teste e os escreve em um arquivo.

  Argumentos:
    num_rows: Número de linhas a serem geradas.

  Retorno:
    None.
  """
  with open("data/measurements1.txt", "w", encoding="utf-8") as file:
    for _ in tqdm(range(num_rows)):
      # Gera dados aleatórios
      station_name = fake.city()
      temperature = round(fake.random_element([random.uniform(-50, 50), random.uniform(50, 100)]), 1)

      # Escreve a linha no arquivo
      file.write(f"{station_name};{temperature}\n")

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Gera os dados de teste
start_time = time.time()
generate_test_data(num_rows)
end_time = time.time()

# Exibe o tempo real de geração
tempo_em_segundos = end_time - start_time
print(f"Tempo real de geração:{format_time(tempo_em_segundos)}")

# Converte o tamanho do arquivo para megabytes
file_size = os.path.getsize("data/measurements1.txt")
file_size_mb = file_size / (1024 * 1024)

# Exibe o tamanho do arquivo em megabytes
print(f"Tamanho real do arquivo: {file_size_mb:.2f} Megabytes")

Digite o número de linhas a serem geradas:  1000000


100%|██████████████████████████████████████████| 1000000/1000000 [01:57<00:00, 8520.61it/s]

Tempo real de geração:1 minutos 57 segundos
Tamanho real do arquivo: 17.36 Megabytes


## Modelo para gerar dados aleatórios com faker e multiprocessing.

In [8]:
import faker
import os
import sys
import time
import random
from tqdm import tqdm
from multiprocessing import Pool

# Cria um objeto Faker
fake = faker.Faker()

# Formata o tempo.
def format_time(segundos):
    """
    Formata os milisegundos em hora:minuto:segundo
    """
    if segundos < 60:
        return f"{segundos:.3f} segundos"
    elif segundos < 3600:
        minutos, segundos = divmod(segundos, 60)
        return f"{int(minutos)} minutos {int(segundos)} segundos"
    else:
        horas, remainder = divmod(seconds, 3600)
        minutos, segundos = divmod(remainder, 60)
        if minutos == 0:
            return f"{int(horas)} horas {int(segundos)} segundos"
        else:
            return f"{int(horas)} horas {int(minutos)} minutos {int(segundos)} segundos"

# Função para gerar dados de teste
def generate_test_data(num_rows, chunk_size):
  """
  Gera dados de teste e os escreve em um arquivo.

  Argumentos:
    num_rows: Número de linhas a serem geradas.
    chunk_size: Tamanho do bloco de dados a ser gerado por cada thread.

  Retorno:
    None.
  """
  with open("data/measurements.txt", "w", encoding="utf-8") as file:
    for i in tqdm(range(0, num_rows, chunk_size)):
      # Gera um bloco de dados
      data_chunk = []
      for _ in range(chunk_size):
        station_name = fake.city()
        temperature = round(fake.random_element([random.uniform(-50, 50), random.uniform(50, 100)]), 1)
        data_chunk.append(f"{station_name};{temperature}\n")

      # Escreve o bloco de dados no arquivo
      file.writelines(data_chunk)

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Define o tamanho do bloco de dados
chunk_size = 10_000  # Ajuste este valor de acordo com a sua memória disponível

# Número de threads
num_workers = os.cpu_count()

# Gera os dados de teste em paralelo
start_time = time.time()
with Pool(num_workers) as pool:
  pool.starmap(generate_test_data, [(num_rows, chunk_size)])
end_time = time.time()

# Exibe o tempo real de geração
tempo_em_segundos = end_time - start_time
print(f"Tempo real de geração:{format_time(tempo_em_segundos)}")

# Exibe o tamanho real do arquivo
file_size = os.path.getsize("data/measurements.txt")

# Converte o tamanho do arquivo para megabytes
file_size_mb = file_size / (1024 * 1024)

# Exibe o tamanho do arquivo em megabytes
print(f"Tamanho real do arquivo: {file_size_mb:.2f} Megabytes")

Digite o número de linhas a serem geradas:  1000000


100%|████████████████████████████████████████████████████| 100/100 [01:46<00:00,  1.07s/it]


Tempo real de geração:1 minutos 47 segundos
Tamanho real do arquivo: 17.36 Megabytes


## Scrip python para gerar dados aleatórios
* **O script foi retirado do desafio [The One Billion Row Challenge](https://github.com/gunnarmorling/1brc), originalmente proposto para Java.**

In [5]:
# Based on https://github.com/gunnarmorling/1brc/blob/main/src/main/java/dev/morling/onebrc/CreateMeasurements.java

import os
import sys
import random
import time


def check_args(file_args):
    """
    Sanity checks out input and prints out usage if input is not a positive integer
    """
    try:
        if len(file_args) != 2 or int(file_args[1]) <= 0:
            raise Exception()
    except:
        print("Usage:  create_measurements.sh <positive integer number of records to create>")
        print("        You can use underscore notation for large number of records.")
        print("        For example:  1_000_000_000 for one billion")
        exit()


def build_weather_station_name_list():
    """
    Grabs the weather station names from example data provided in repo and dedups
    """
    station_names = []
    with open('data/weather_stations.csv', 'r') as file:
        file_contents = file.read()
    for station in file_contents.splitlines():
        if "#" in station:
            next
        else:
            station_names.append(station.split(';')[0])
    return list(set(station_names))


def convert_bytes(num):
    """
    Convert bytes to a human-readable format (e.g., KiB, MiB, GiB)
    """
    for x in ['bytes', 'KiB', 'MiB', 'GiB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def format_elapsed_time(seconds):
    """
    Format elapsed time in a human-readable format
    """
    if seconds < 60:
        return f"{seconds:.3f} seconds"
    elif seconds < 3600:
        minutes, seconds = divmod(seconds, 60)
        return f"{int(minutes)} minutes {int(seconds)} seconds"
    else:
        hours, remainder = divmod(seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        if minutes == 0:
            return f"{int(hours)} hours {int(seconds)} seconds"
        else:
            return f"{int(hours)} hours {int(minutes)} minutes {int(seconds)} seconds"


def estimate_file_size(weather_station_names, num_rows_to_create):
    """
    Tries to estimate how large a file the test data will be
    """
    total_name_bytes = sum(len(s.encode("utf-8")) for s in weather_station_names)
    avg_name_bytes = total_name_bytes / float(len(weather_station_names))

    # avg_temp_bytes = sum(len(str(n / 10.0)) for n in range(-999, 1000)) / 1999
    avg_temp_bytes = 4.400200100050025

    # add 2 for separator and newline
    avg_line_length = avg_name_bytes + avg_temp_bytes + 2

    human_file_size = convert_bytes(num_rows_to_create * avg_line_length)

    return f"Estimated max file size is:  {human_file_size}."


def build_test_data(weather_station_names, num_rows_to_create):
    """
    Generates and writes to file the requested length of test data
    """
    start_time = time.time()
    coldest_temp = -99.9
    hottest_temp = 99.9
    station_names_10k_max = random.choices(weather_station_names, k=10_000)
    batch_size = 10000 # instead of writing line by line to file, process a batch of stations and put it to disk
    chunks = num_rows_to_create // batch_size
    print('Building test data...')

    try:
        with open("data/measurements1.txt", 'w') as file:
            progress = 0
            for chunk in range(chunks):
                
                batch = random.choices(station_names_10k_max, k=batch_size)
                prepped_deviated_batch = '\n'.join([f"{station};{random.uniform(coldest_temp, hottest_temp):.1f}" for station in batch]) # :.1f should quicker than round on a large scale, because round utilizes mathematical operation
                file.write(prepped_deviated_batch + '\n')
                
                # Update progress bar every 1%
                if (chunk + 1) * 100 // chunks != progress:
                    progress = (chunk + 1) * 100 // chunks
                    bars = '=' * (progress // 2)
                    sys.stdout.write(f"\r[{bars:<50}] {progress}%")
                    sys.stdout.flush()
        sys.stdout.write('\n')
    except Exception as e:
        print("Something went wrong. Printing error info and exiting...")
        print(e)
        exit()
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    file_size = os.path.getsize("data/measurements1.txt")
    human_file_size = convert_bytes(file_size)
 
    print("Test data successfully written to data/measurements1.txt")
    print(f"Actual file size:  {human_file_size}")
    print(f"Elapsed time: {format_elapsed_time(elapsed_time)}")


def main():
    """
    main program function
    """
    num_rows_to_create = 1_000_000
    weather_station_names = []
    weather_station_names = build_weather_station_name_list()
    print(estimate_file_size(weather_station_names, num_rows_to_create))
    build_test_data(weather_station_names, num_rows_to_create)
    print("Test data build complete.")


if __name__ == "__main__":
    main()
exit()

### Conclusão
* **O modelo para gerar dados com o faker se saiu extremamente lento em comparação ao script do [Gunnar Morling](https://github.com/gunnarmorling) acima.**

## Gerando 1 bilhão de linhas:

In [6]:
# gerando 1 bilhão de linhas
num_rows_to_create = 1_000_000_000
weather_station_names = []
weather_station_names = build_weather_station_name_list()
print(estimate_file_size(weather_station_names, num_rows_to_create))
build_test_data(weather_station_names, num_rows_to_create)
print("Test data build complete.")




Estimated max file size is:  14.8 GiB.
Building test data...
[==================================================] 100%
Test data successfully written to data/measurements.txt
Actual file size:  14.9 GiB
Elapsed time: 15 minutes 3 seconds
Test data build complete.


## Decoradores

In [1]:
# Criando um decorator para calcular o tempo de processamento
def timer(func):
    import time
    # Formata o tempo.
    def format_time(segundos: int): 
        """
        Formata os milisegundos em hora:minuto:segundo
        """
        if segundos < 60:
            return f"{segundos:.3f} segundos"
        elif segundos < 3600:
            minutos, segundos = divmod(segundos, 60)
            return f"{int(minutos)} minutos {int(segundos)} segundos"
        else:
            horas, remainder = divmod(seconds, 3600)
            minutos, segundos = divmod(remainder, 60)
            if minutos == 0:
                return f"{int(horas)} horas {int(segundos)} segundos"
            else:
                return f"{int(horas)} horas {int(minutos)} minutos {int(segundos)} segundos"  

    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        tempo_em_segundos = end - start       
        # Exibe o tempo real de geração    
        print(f"{func.__name__} Tempo de processamento:{format_time(tempo_em_segundos)}")   
        #print(type(format_time(tempo_em_segundos)))
        return result
    return wrapper


## Pandas - min, max e mean em 1 bilhão de linhas

In [ ]:
# Instalando a lib pandas.
!poetry add pandas

In [1]:
# Script para min, max e mean de um bilhão de linhas com pandas.
import pandas as pd
import time
from multiprocessing import Pool, cpu_count
from tqdm import tqdm  # importa o tqdm para barra de progresso

CONCURRENCY = cpu_count()

total_linhas = 1_000_000_000  # Total de linhas conhecido
chunksize = 100_000_000  # Define o tamanho do chunk
filename = "data/measurements.txt"  # Certifique-se de que este é o caminho correto para o arquivo

# Formata o tempo.
def format_time(segundos):
    """
    Formata os milisegundos em hora:minuto:segundo
    """
    if segundos < 60:
        return f"{segundos:.3f} segundos"
    elif segundos < 3600:
        minutos, segundos = divmod(segundos, 60)
        return f"{int(minutos)} minutos {int(segundos)} segundos"
    else:
        horas, remainder = divmod(seconds, 3600)
        minutos, segundos = divmod(remainder, 60)
        if minutos == 0:
            return f"{int(horas)} horas {int(segundos)} segundos"
        else:
            return f"{int(horas)} horas {int(minutos)} minutos {int(segundos)} segundos"

def process_chunk(chunk):
    # Agrega os dados dentro do chunk usando Pandas
    aggregated = chunk.groupby('station')['measure'].agg(['min', 'max', 'mean']).reset_index()
    return aggregated

def create_df_with_pandas(filename, total_linhas, chunksize=chunksize):
    total_chunks = total_linhas // chunksize + (1 if total_linhas % chunksize else 0)
    results = []

    with pd.read_csv(filename, sep=';', header=None, names=['station', 'measure'], chunksize=chunksize) as reader:
        # Envolvendo o iterador com tqdm para visualizar o progresso
        with Pool(CONCURRENCY) as pool:
            for chunk in tqdm(reader, total=total_chunks, desc="Processando"):
                # Processa cada chunk em paralelo
                result = pool.apply_async(process_chunk, (chunk,))
                results.append(result)

            results = [result.get() for result in results]

    final_df = pd.concat(results, ignore_index=True)

    final_aggregated_df = final_df.groupby('station').agg({
        'min': 'min',
        'max': 'max',
        'mean': 'mean'
    }).reset_index().sort_values('station')

    return final_aggregated_df

if __name__ == "__main__":
    import time

    print("Iniciando o processamento do arquivo.")
    start_time = time.time()
    df = create_df_with_pandas(filename, total_linhas, chunksize)
    end_time = time.time()
    tempo_em_segundos = end_time - start_time
    print(df.head())
    # Exibe o tempo real de geração    
    print(f"Tempo de processamento:{format_time(tempo_em_segundos)}")


Iniciando o processamento do arquivo.


Processando: 100%|█████████████████████████████████████████| 10/10 [09:20<00:00, 56.07s/it]


       station   min   max      mean
0     Aabenraa -99.9  99.9 -0.061535
1       Aalten -99.9  99.9 -0.161205
2    Abadiânia -99.9  99.9 -0.051514
3     Abalessa -99.9  99.9  0.211034
4  Abangaritos -99.9  99.9  0.066196
Tempo de processamento:-1709939632.573 segundos


## Polars - min, max e mean em 1 bilhão de linhas

In [ ]:
# Instalando a lib polars
!poetry add polars

In [3]:
import polars as pl
@timer
def create_polars_df(): 
    pl.Config.set_streaming_chunk_size(5000000)
    # Leitura do arquivo CSV e definição do schema
    return (pl.scan_csv("data/measurements.txt", separator=";", has_header=False,
                        schema={"station": pl.String, "measure": pl.Float64})
                        .group_by("station").agg(
                                                 max_temp=pl.col("measure").max(),
                                                 min_temp=pl.col("measure").min(),
                                                 mean_temp=pl.col("measure").mean()
                                                ).sort("station").collect(streaming=True)
           )   
if __name__ == "__main__":    
    df = create_polars_df()
    print(df)
   


create_polars_df Tempo de processamento:26.558 segundos
shape: (8_836, 4)
┌────────────────┬──────────┬──────────┬───────────┐
│ station        ┆ max_temp ┆ min_temp ┆ mean_temp │
│ ---            ┆ ---      ┆ ---      ┆ ---       │
│ str            ┆ f64      ┆ f64      ┆ f64       │
╞════════════════╪══════════╪══════════╪═══════════╡
│ Aabenraa       ┆ 99.9     ┆ -99.9    ┆ -0.062295 │
│ Aalten         ┆ 99.9     ┆ -99.9    ┆ -0.161752 │
│ Abadiânia      ┆ 99.9     ┆ -99.9    ┆ -0.052772 │
│ Abalessa       ┆ 99.9     ┆ -99.9    ┆ 0.211089  │
│ Abangaritos    ┆ 99.9     ┆ -99.9    ┆ 0.063772  │
│ …              ┆ …        ┆ …        ┆ …         │
│ ’Aïn Abessa    ┆ 99.9     ┆ -99.9    ┆ -0.013338 │
│ ’Aïn Azel      ┆ 99.9     ┆ -99.9    ┆ -0.040762 │
│ ’Aïn Roua      ┆ 99.9     ┆ -99.9    ┆ -0.026898 │
│ ’s-Gravenzande ┆ 99.9     ┆ -99.9    ┆ -0.080973 │
│ ’s-Heerenberg  ┆ 99.9     ┆ -99.9    ┆ 0.168265  │
└────────────────┴──────────┴──────────┴───────────┘


## Duckdb - min, max e mean em 1 bilhão de linhas

In [ ]:
!poetry add duckdb

In [7]:
import duckdb
@timer
def create_duckdb():
    duckdb.sql("""
        SELECT station,
            MIN(measure) AS min_temperature,
            CAST(AVG(measure) AS DECIMAL()) AS mean_temperature,
            MAX(measure) AS max_temperature
        FROM read_csv("data/measurements.txt", AUTO_DETECT=FALSE, sep=';', columns={'station':VARCHAR, 'measure': 'DECIMAL'})
        GROUP BY station
        ORDER BY station
    """).show()

if __name__ == "__main__":
    create_duckdb()   

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────────┬─────────────────┬──────────────────┬─────────────────┐
│     station     │ min_temperature │ mean_temperature │ max_temperature │
│     varchar     │  decimal(18,3)  │  decimal(18,3)   │  decimal(18,3)  │
├─────────────────┼─────────────────┼──────────────────┼─────────────────┤
│ Aabenraa        │         -99.900 │           -0.062 │          99.900 │
│ Aalten          │         -99.900 │           -0.162 │          99.900 │
│ Abadiânia       │         -99.900 │           -0.053 │          99.900 │
│ Abalessa        │         -99.900 │            0.211 │          99.900 │
│ Abangaritos     │         -99.900 │            0.064 │          99.900 │
│ Abano Terme     │         -99.900 │            0.172 │          99.900 │
│ Abaré           │         -99.900 │           -0.267 │          99.900 │
│ Abbeville       │         -99.900 │            0.074 │          99.900 │
│ Abbiategrasso   │         -99.900 │           -0.195 │          99.900 │
│ Abbots Langley  │      

In [9]:
import duckdb
@timer
def create_duckdb(): 
    conn = duckdb.connect(':memory:')
    start_time = time.time()
    display(conn.execute("""
            SELECT station,
                MIN(measure) AS min_temperature,
                CAST(AVG(measure) AS DECIMAL()) AS mean_temperature,
                MAX(measure) AS max_temperature
            FROM read_csv("data/measurements.txt", AUTO_DETECT=FALSE, sep=';', columns={'station':VARCHAR, 'measure': 'DECIMAL'})
            GROUP BY station
            ORDER BY station
        """).df())
    
if __name__ == "__main__":
    create_duckdb()   

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,station,min_temperature,mean_temperature,max_temperature
0,Aabenraa,-99.9,-0.062,99.9
1,Aalten,-99.9,-0.162,99.9
2,Abadiânia,-99.9,-0.053,99.9
3,Abalessa,-99.9,0.211,99.9
4,Abangaritos,-99.9,0.064,99.9
...,...,...,...,...
8831,’Aïn Abessa,-99.9,-0.013,99.9
8832,’Aïn Azel,-99.9,-0.041,99.9
8833,’Aïn Roua,-99.9,-0.027,99.9
8834,’s-Gravenzande,-99.9,-0.081,99.9


create_duckdb Tempo de processamento:19.675 segundos


In [9]:
display(conn.execute('SHOW TABLES').df())

,name
0,measurements


In [10]:
display(conn.execute('Select * from measurements limit 10').df())

,station,measure
0,Uppalapādu,-29.9
1,Kaman,-34.0
2,Tamu,-40.2
3,Quillabamba,50.6
4,Jabuticabal,15.8
5,Kilibo,-98.4
6,Tillaivilāgam,48.3
7,Ensenada,97.4
8,Indio,64.6
9,Buxton,-54.7


In [30]:
conn.close()

## Dask - min, max e mean em 1 bilhão de linhas

In [ ]:
#necessário Instalar.
!pip install dask-expr

In [18]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
@timer
def create_dask():    
    # Ler o arquivo txt diretamente em um DataFrame Dask
    df = dd.read_csv('data/measurements.txt', delimiter=';', header=None, names=['station', 'measure'])
    # min, max, e mean pela cidade ordenado pelo index
    return display(df.groupby('station').agg({'measure': ['max','min','mean']}).compute().sort_index())

if __name__ == "__main__":
    create_dask()  


measure                
                   max   min      mean
station                               
Aabenraa          99.9 -99.9 -0.062295
Aalten            99.9 -99.9 -0.161752
Abadiânia         99.9 -99.9 -0.052772
Abalessa          99.9 -99.9  0.211089
Abangaritos       99.9 -99.9  0.063772
...                ...   ...       ...
’Aïn Abessa       99.9 -99.9 -0.013338
’Aïn Azel         99.9 -99.9 -0.040762
’Aïn Roua         99.9 -99.9 -0.026898
’s-Gravenzande    99.9 -99.9 -0.080973
’s-Heerenberg     99.9 -99.9  0.168265

[8836 rows x 3 columns]

create_dask Tempo de processamento:7 minutos 41 segundos


In [13]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
@timer
def create_dask():    
    # Ler o arquivo txt diretamente em um DataFrame Dask
    df = dd.read_csv('data/measurements.txt', delimiter=';', header=None, names=['station', 'measure'])
    # min, max, e mean pela cidade ordenado pelo index
    print(df.groupby('station').
    agg({'measure': ['max','min','mean']}).
    compute().
    sort_index())
if __name__ == "__main__":
    create_dask() 
    

               measure                
                   max   min      mean
station                               
Aabenraa          99.9 -99.9 -0.062295
Aalten            99.9 -99.9 -0.161752
Abadiânia         99.9 -99.9 -0.052772
Abalessa          99.9 -99.9  0.211089
Abangaritos       99.9 -99.9  0.063772
...                ...   ...       ...
’Aïn Abessa       99.9 -99.9 -0.013338
’Aïn Azel         99.9 -99.9 -0.040762
’Aïn Roua         99.9 -99.9 -0.026898
’s-Gravenzande    99.9 -99.9 -0.080973
’s-Heerenberg     99.9 -99.9  0.168265

[8836 rows x 3 columns]
create_dask Tempo de processamento:7 minutos 47 segundos


## Pyspark - min, max e mean em 1 bilhão de linhas.

In [ ]:
#instalação da lib
!poetry add pyspark

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min as spark_min, max as spark_max, avg as spark_avg
@timer
def create_pyspark():     
    # Inicializar uma sessão Spark
    spark = SparkSession.builder \
        .appName("Temperature Analysis") \
        .getOrCreate()
    
    # Ler o arquivo CSV diretamente em um DataFrame Spark
    df = spark.read.option("header", "false").option("delimiter", ";").csv("data/measurements.txt") \
        .toDF("City", "Temperature")
    
    # Converter a coluna 'Temperature' para tipo numérico
    df = df.withColumn("Temperature", col("Temperature").cast("float"))
    
    # Calcular estatísticas usando Spark SQL
    statistics = df.groupBy("City") \
        .agg(spark_min("Temperature").alias("Min Temperature"),
             spark_max("Temperature").alias("Max Temperature"),
             spark_avg("Temperature").alias("Avg Temperature"))
    
    # Ordenar as estatísticas pela cidade
    statistics_sorted = statistics.orderBy("City")
    
    # Mostrar as estatísticas
    return statistics_sorted.show()
    
    # Encerrar a sessão Spark
    # spark.stop()

if __name__ == "__main__":
    create_pyspark()  


+--------------+---------------+---------------+--------------------+
|          City|Min Temperature|Max Temperature|     Avg Temperature|
+--------------+---------------+---------------+--------------------+
|      Aabenraa|          -99.9|           99.9| -0.0622947488794981|
|        Aalten|          -99.9|           99.9|-0.16175193009676553|
|     Abadiânia|          -99.9|           99.9|-0.05277215372419822|
|      Abalessa|          -99.9|           99.9| 0.21108941553593294|
|   Abangaritos|          -99.9|           99.9| 0.06377211310947027|
|   Abano Terme|          -99.9|           99.9| 0.17194244350252394|
|         Abaré|          -99.9|           99.9| -0.2666045273852838|
|     Abbeville|          -99.9|           99.9| 0.07425508222933982|
| Abbiategrasso|          -99.9|           99.9|-0.19547850239845785|
|Abbots Langley|          -99.9|           99.9| 0.20737068497627278|
|      Abdulino|          -99.9|           99.9|  0.1269626020386111|
|      Aberdare|    

## Vaex - min, max e mean em 1 bilhão de linhas.

* **Para utilizar o Vaex foi necessário selecionar outro kernel com python 3.10.13.**

In [2]:
!python --version

Python 3.10.13


In [4]:
import vaex
import time

def main(filename):
    start_time = time.time()

    # Leitura do arquivo CSV utilizando Vaex
    df = vaex.from_csv(filename, names=['city', 'temperature'], sep=';')

    # Cálculo das estatísticas
    min_temperature = df.groupby(df['city']).agg({'temperature': 'min'})
    max_temperature = df.groupby(df['city']).agg({'temperature': 'max'})
    mean_temperature = df.groupby(df['city']).agg({'temperature': 'mean'})

    # Concatenação dos resultados
    combined_results = vaex.concat([min_temperature, max_temperature, mean_temperature])
    combined_results.columns = ['min_temperature', 'max_temperature', 'mean_temperature']

    # Exibição dos resultados
    print(combined_results)

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {format_time(execution_time)}")

if __name__ == "__main__":
    filename = "data/measurements1.txt"
    main(filename)


#        city                temperature
0        West Pamela         -44.1
1        New Ashley          -47.9
2        Port Cody           -46.8
3        Christianfurt       -42.9
4        South Debbietown    17.5
...      ...                 ...
274,081  North Rosestad      67.1
274,082  South Blakeborough  46.2
274,083  Port Normachester   -5.1
274,084  South Kentbury      -27.3
274,085  North Katrinaville  67.8
Execution time: 00:00:01.49


In [42]:
import vaex
@timer
def create_vaex(filename):
    # Leitura do arquivo CSV utilizando Vaex
    df = vaex.from_csv(filename, names=['station', 'measure'], sep=';')

    # Cálculo das estatísticas
    combined_results = df.groupby(df['station']).agg({'measure': ['min', 'max', 'mean']})

    # Exibição dos resultados
    return display(combined_results)
   
if __name__ == "__main__":
    filename = "data/measurements1.txt"
    create_vaex(filename)


#,station,measure_min,measure_max,measure_mean
0,West Pamela,-44.1,95.4,38.80804597701149
1,New Ashley,-47.9,99.8,39.13777777777778
2,Port Cody,-46.8,99.0,37.87761194029851
3,Christianfurt,-42.9,99.7,46.25333333333334
4,South Debbietown,17.5,99.1,56.633333333333326
...,...,...,...,...
"91,357",West Lawrencestad,63.4,67.8,65.6
"91,358",East Tomburgh,30.9,30.9,30.9
"91,359",West Savannahfurt,66.4,66.4,66.4
"91,360",Cristianberg,2.0,2.0,2.0


create_vaex Tempo de processamento:1.453 segundos


In [47]:
df = vaex.from_csv('data/measurements.txt', names=['station', 'measure'], sep=';',convert=True,chunk_size=10_000_000 , progress=True)

Converting csv to chunk files
Saved chunk #0 to data/measurements.txt_chunk_0.hdf5
Saved chunk #1 to data/measurements.txt_chunk_1.hdf5
Saved chunk #2 to data/measurements.txt_chunk_2.hdf5
Saved chunk #3 to data/measurements.txt_chunk_3.hdf5
Saved chunk #4 to data/measurements.txt_chunk_4.hdf5
Saved chunk #5 to data/measurements.txt_chunk_5.hdf5
Saved chunk #6 to data/measurements.txt_chunk_6.hdf5
Saved chunk #7 to data/measurements.txt_chunk_7.hdf5
Saved chunk #8 to data/measurements.txt_chunk_8.hdf5
Saved chunk #9 to data/measurements.txt_chunk_9.hdf5
Saved chunk #10 to data/measurements.txt_chunk_10.hdf5
Saved chunk #11 to data/measurements.txt_chunk_11.hdf5
Saved chunk #12 to data/measurements.txt_chunk_12.hdf5
Saved chunk #13 to data/measurements.txt_chunk_13.hdf5
Saved chunk #14 to data/measurements.txt_chunk_14.hdf5
Saved chunk #15 to data/measurements.txt_chunk_15.hdf5
Saved chunk #16 to data/measurements.txt_chunk_16.hdf5
Saved chunk #17 to data/measurements.txt_chunk_17.hdf5


In [50]:
@timer
def test_vaex():
    return df.groupby(df['station']).agg({'measure': ['min', 'max', 'mean']})


test_vaex()

test_vaex Tempo de processamento:49.093 segundos


#,station,measure_min,measure_max,measure_mean
0,Borgentreich,-99.9,99.9,-0.17837456749134986
1,Falkenberg,-99.9,99.9,-0.15636033723972742
2,Workington,-99.9,99.9,0.1779083903507457
3,Al Wajh,-99.9,99.9,-0.3690835063228942
4,Murgap,-99.9,99.9,-0.052552561593657586
...,...,...,...,...
"8,831",Tettu,-99.9,99.9,0.3071891767668189
"8,832",Belm,-99.9,99.9,-0.21524021476617236
"8,833",Nyíregyháza,-99.9,99.9,-0.13624434888080206
"8,834",Sherpur,-99.9,99.9,-0.36114642043410505


In [ ]:
<a id="ancora01"></a>

In [ ]:
<a id="ancora02"></a>

In [ ]:
<a id="ancora03"></a>

In [ ]:
<a id="ancora04"></a>

In [ ]:
<a id="ancora05"></a>

In [ ]:
<a id="ancora06"></a>

In [ ]:
<a id="ancora07"></a>

In [ ]:
<a id="ancora08"></a>

In [ ]:
<a id="ancora09"></a>

In [ ]:
<a id="ancora10"></a>

In [ ]:
<a id="ancora11"></a>

In [ ]:
<a id="ancora12"></a>

In [ ]:
<a id="ancora13"></a>

In [ ]:
<a id="ancora14"></a>

In [ ]:
<a id="ancora15"></a>

In [ ]:
<a id="ancora16"></a>

In [ ]:
<a id="ancora17"></a>

In [ ]:
<a id="ancora18"></a>

In [ ]:
<a id="ancora19"></a>

In [ ]:
<a id="ancora20"></a>